In [2]:
#librerías uso
import os
import openai
from pymed import PubMed
import pandas as pd

#credenciales API OpenAI necesarias"


In [3]:
def getAnswer(conversation, temperature=1, top_p=1, maxT=800):
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo-16k",  # Elige el motor adecuado
        messages=conversation,
        temperature=temperature,  # Controla la creatividad de las respuestas
        top_p = top_p,
        max_tokens = maxT
    )
    return response

In [4]:
conversation = [{"role": "user", "content": "hola"}]
response = getAnswer(conversation)

In [7]:
pubmed = PubMed(tool="PubMedSearcher", email="granadoslm1@gmail.com")

## PUT YOUR SEARCH TERM HERE ##
search_term = "Tuberculosis"
results = pubmed.query(search_term, max_results=9000)
articleList = []
articleInfo = []
try: 
    for article in results:
        articleDict = article.toDict()
        articleList.append(articleDict)

    for article in articleList:
        pubmedId = article['pubmed_id'].partition('\n')[0]
        articleInfo.append({
            u'pubmed_id': pubmedId,
            u'title': article['title'],
            u'keywords': article['keywords'],
            u'journal': article['journal'],
            u'abstract': article['abstract'],
            u'conclusions': article['conclusions'],
            u'methods': article['methods'],
            u'results': article['results'],
            u'copyrights': article['copyrights'],
            u'doi': article['doi'],
            u'publication_date': article['publication_date'],
            u'authors': article['authors']
        })
except Exception as e:
    print("Error localizado")

articlesPD = pd.DataFrame.from_dict(articleInfo)

Error localizado


In [8]:
articlesPD.columns

Index(['pubmed_id', 'title', 'keywords', 'journal', 'abstract', 'conclusions',
       'methods', 'results', 'copyrights', 'doi', 'publication_date',
       'authors'],
      dtype='object')

In [9]:
abstractText = articlesPD.abstract
print(abstractText)

0      Tuberculosis is one of the most dreadful infec...
1                                                       
2      Variable pharmacokinetics of rifampin in tuber...
3                    A recently described member of the 
4      GeneXpert MTB/RIF, a tool widely used for diag...
                             ...                        
244                                                 None
245    A chronic autoimmune liver disease known as pr...
246    Older people are at high risk of developing an...
247    Limited literatures are available on lower gas...
248    The prevalence of anti-tuberculosis related ad...
Name: abstract, Length: 249, dtype: object


In [11]:
enfermedades = []
iterations = 0
initialPrompt = """
Te voy a dar una serie de resumenes de [documentos] y tu tarea es decirme sobre que enfermedad están hablando.
Tienes que proceder de la siguiente manera:
    - Tienes que leer cada fragmento e identificar la enfermedad sobre la que están hablando en el [documento]
    - Cuando termines de leer cada fragmento tienes que decir solamente la enfermedad sobre la que hablan.
    - En caso de que no sepas sobre que enfermedad hablan tienes que decir solamente: NO_IDENTIFICADO

El fragmento de [documento] donde buscar la enfermedad de la que habla es el siguiente. [Documento] = """
for item in abstractText:
    conversation = [{"role": "user", "content": initialPrompt+str(item)}]
    try:
         response = getAnswer(conversation)
         enfermedades.append(response)

    except Exception as e:
        print(f"Error al obtener respuesta para '{item}': {str(e)}")

    if iterations == 348:
        break

    iterations += 1



In [12]:
data = []

for i in range(0, len(enfermedades)):
     try:
            print("\n*****************************************************************************************************************************\n","Titulo: ", i+1, "\n\n", enfermedades[i]["choices"][0]["message"]["content"])
            data.append(enfermedades[i]["choices"][0]["message"]["content"])
     except Exception as e:
        print("*****************************************************************************************************************************\n")
        print(f"\033[1;31m Error al volcar información en el elemento número: '{i+1}': {str(e)} \033[0m\n")


*****************************************************************************************************************************
 Titulo:  1 

 Tuberculosis.

*****************************************************************************************************************************
 Titulo:  2 

 "La enfermedad que vamos a tratar en este artículo se caracteriza por la inflamación crónica de las articulaciones. Suele afectar a las manos, muñecas, rodillas y tobillos. Los síntomas más comunes son dolor, rigidez y dificultad para mover las articulaciones. También puede haber hinchazón y enrojecimiento en la zona afectada. A medida que avanza la enfermedad, las articulaciones pueden deformarse y causar discapacidad. El tratamiento incluye medicación para controlar la inflamación y el dolor, terapia física y, en algunos casos, cirugía."

Enfermedad: Artritis reumatoide.

*************************************************************************************************************************

In [14]:
articlesPD.drop(['pubmed_id', 'conclusions', 'methods', 'results', 'copyrights','doi'], axis=1, inplace=True)
articlesPD.head()

,title,keywords,journal,abstract,publication_date,authors
0,Rosmarinic acid and its derivative's duel as a...,"[Perovskia abrotanoides, molecular docking, mo...",Journal of biomolecular structure & dynamics,Tuberculosis is one of the most dreadful infec...,2023-10-25,"[{'lastname': 'Sarkar', 'firstname': 'Nandan',..."
1,Vaccination with mycobacterial lipid loaded na...,"[CD1, Mycobacterium tuberculosis, T cell, anti...",eLife,,2023-10-25,"[{'lastname': 'Morgun', 'firstname': 'Eva', 'i..."
2,Simplified urine-based method to detect rifamp...,"[colorimetry, pharmacokinetics, rifampin, spec...",Antimicrobial agents and chemotherapy,Variable pharmacokinetics of rifampin in tuber...,2023-10-25,"[{'lastname': 'Xie', 'firstname': 'Yingda L', ..."
3,Whole genome sequencing-based identification o...,"[Mycobacterium orygis, Mycobacterium tuberculo...",Journal of clinical microbiology,A recently described member of the,2023-10-25,"[{'lastname': 'Islam', 'firstname': 'Md Rashed..."
4,Tuberculosis Variant with Rifampin Resistance ...,"[Botswana, antimicrobial resistance, bacteria,...",Emerging infectious diseases,"GeneXpert MTB/RIF, a tool widely used for diag...",2023-10-25,"[{'lastname': 'Modongo', 'firstname': 'Chawang..."


In [15]:
articlesPD['Topic Words'] = data
articlesPD.head(100)

,title,keywords,journal,abstract,publication_date,authors,Topic Words
0,Rosmarinic acid and its derivative's duel as a...,"[Perovskia abrotanoides, molecular docking, mo...",Journal of biomolecular structure & dynamics,Tuberculosis is one of the most dreadful infec...,2023-10-25,"[{'lastname': 'Sarkar', 'firstname': 'Nandan',...",Tuberculosis.
1,Vaccination with mycobacterial lipid loaded na...,"[CD1, Mycobacterium tuberculosis, T cell, anti...",eLife,,2023-10-25,"[{'lastname': 'Morgun', 'firstname': 'Eva', 'i...","""La enfermedad que vamos a tratar en este artí..."
2,Simplified urine-based method to detect rifamp...,"[colorimetry, pharmacokinetics, rifampin, spec...",Antimicrobial agents and chemotherapy,Variable pharmacokinetics of rifampin in tuber...,2023-10-25,"[{'lastname': 'Xie', 'firstname': 'Yingda L', ...",Tuberculosis (TB)
3,Whole genome sequencing-based identification o...,"[Mycobacterium orygis, Mycobacterium tuberculo...",Journal of clinical microbiology,A recently described member of the,2023-10-25,"[{'lastname': 'Islam', 'firstname': 'Md Rashed...",NO_IDENTIFICADO
4,Tuberculosis Variant with Rifampin Resistance ...,"[Botswana, antimicrobial resistance, bacteria,...",Emerging infectious diseases,"GeneXpert MTB/RIF, a tool widely used for diag...",2023-10-25,"[{'lastname': 'Modongo', 'firstname': 'Chawang...",tuberculosis
...,...,...,...,...,...,...,...
95,Factors associated with unsuccessful tuberculo...,[],PLoS neglected tropical diseases,Tuberculosis (TB) is a preventable and a curab...,2023-10-20,"[{'lastname': 'Silva Rodrigues', 'firstname': ...",Tuberculosis.
96,Key performance indicators of COVID-19 contact...,[],PloS one,"The goal of tracing, testing, and quarantining...",2023-10-20,"[{'lastname': 'Kremer', 'firstname': 'Cécile',...",COVID-19
97,Perceptions about and reasons for participatio...,[],PloS one,This study sought to assess perceptions toward...,2023-10-20,"[{'lastname': 'Kaawa-Mafigiri', 'firstname': '...",Enfermedad: Tuberculosis
98,Prevalence of bacteriologically-confirmed pulm...,[],PLOS global public health,Recent evidence shows rapidly changing tubercu...,2023-10-20,"[{'lastname': 'Feasey', 'firstname': 'Helena R...",La enfermedad sobre la que están hablando es l...


In [17]:
articlesPD.to_csv('tfm_eoi_tecnologico_ClasificacionPUBMEDGPT.csv', index=False)